In [51]:
from torch import nn

m1 = nn.Module()
m1.foo = nn.Linear(3, 4)
m1

Module(
  (foo): Linear(in_features=3, out_features=4, bias=True)
)

In [52]:
m1.named_children()

<generator object Module.named_children at 0x000001F1B9BBF3D0>

In [53]:

list(m1.named_children())

[('foo', Linear(in_features=3, out_features=4, bias=True))]

In [54]:
list(m1.parameters())

[Parameter containing:
 tensor([[ 0.3389, -0.5633, -0.0506],
         [-0.3285,  0.3248,  0.2828],
         [ 0.3633,  0.0869, -0.3566],
         [-0.2879,  0.5615,  0.3613]], requires_grad=True),
 Parameter containing:
 tensor([0.3018, 0.1970, 0.1302, 0.1104], requires_grad=True)]

In [55]:
class MLP(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)
        self.relu = nn.ReLU()
        
    def forward(self, x): 
        return self.l2(self.relu(self.l1(x)))

In [56]:
model = MLP(784, 50, 10)
model.l1

Linear(in_features=784, out_features=50, bias=True)

In [57]:
model.parameters()

<generator object Module.parameters at 0x000001F1B03E8E40>

In [58]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0152,  0.0088,  0.0257,  ..., -0.0119, -0.0139, -0.0276],
         [-0.0299,  0.0250, -0.0192,  ..., -0.0056,  0.0207, -0.0260],
         [ 0.0064,  0.0117,  0.0164,  ...,  0.0305, -0.0239, -0.0340],
         ...,
         [ 0.0149, -0.0287, -0.0204,  ...,  0.0040, -0.0292,  0.0150],
         [ 0.0161,  0.0102,  0.0259,  ...,  0.0127, -0.0096, -0.0347],
         [-0.0221,  0.0176,  0.0115,  ..., -0.0347,  0.0104,  0.0325]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0081,  0.0046, -0.0340, -0.0325, -0.0145, -0.0154, -0.0011,  0.0184,
          0.0164,  0.0301,  0.0237, -0.0128, -0.0261, -0.0073, -0.0167,  0.0186,
          0.0325,  0.0026, -0.0186,  0.0136, -0.0194, -0.0275, -0.0006, -0.0283,
         -0.0261,  0.0271, -0.0129, -0.0116, -0.0056,  0.0128, -0.0128,  0.0151,
          0.0098,  0.0308,  0.0239, -0.0303, -0.0159,  0.0323,  0.0206,  0.0293,
          0.0199,  0.0052, -0.0122, -0.0317, -0.0200,  0.0240, -0.0109, -0.0143

In [59]:
model

MLP(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=10, bias=True)
  (relu): ReLU()
)

In [60]:
# __setattr__ - whenever you set an attribut it will call this value
# passsing in the key and value
# if it does not start an underscore it will put this value into the modules dictionary with this key
# we can make sure 
# if not k.startswith("_")

# allows us to loop through all of the modules that we have kept stored
def parameters(self):
    for l in self._modules.values(): 
        yield from l.parameters()

In [61]:
class MyModule:
    def __init__(self, n_in, nh, n_out):
        self._modules = {}
        self.l1 = nn.Linear(n_in,nh)
        self.l2 = nn.Linear(nh,n_out)

    def __setattr__(self,k,v):
        if not k.startswith("_"): self._modules[k] = v
        super().__setattr__(k,v)

    def __repr__(self): return f'{self._modules}'
    
    def parameters(self):
        for l in self._modules.values(): yield from l.parameters()

In [62]:
class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l)

    def forward(self, x): 
        # reduce applies the function to the initial value initially but then
        # uses the new output as the new input value and continues applying the list of layers
        return reduce(lambda val, layer: layer(val), self.layers, x)
     
# Now with our updates module version we can now enumerate all layers and register every single module to make it very easy

In [63]:
# Instead of using a model we can easily use the sequential model taht takes in sequential layers that take advantage of nn.ModuleList

class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def forward(self, x):
        for l in self.layers: x = l(x)
        return x

In [64]:
# optim - optimiser

class Optimizer():
    def __init__(self, params, lr=0.5): 
        self.params, self.lr = list(params), lr

    def step(self):
        with torch.no_grad():
            for p in self.params: 
                p -= p.grad * self.lr

    def zero_grad(self):
        for p in self.params: 
            p.grad.data.zero_()
     

In [65]:
from torch import optim
from torch import nn

model = nn.Sequential(nn.Linear(784, 50), nn.ReLU(), nn.Linear(50, 10))
optimiser = optim.SGD(model.parameters(), lr = 0.1)

for epoch in range(epochs):
    for i in range(0, n, bs):
        # take a slice from i -> (i + bs | n) since we can be at the end
        s = slice(i, min(n, i + bs))
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()
    report(loss, preds, yb)

NameError: name 'epochs' is not defined

In [71]:
# A dataset allows us to convert these two lines of code from
s = slice(i, min(n, i + bs))
xb, yb = x_train[s], y_train[s]

# into 

xb, yb = train_ds[i : min(n, i+bs)]

NameError: name 'i' is not defined

In [72]:
# A dataloader will allow us to store the dataset and then loop through the data and provide our slices of data via minibatches

for i in range(0, n, bs):
    xb, yb = train_ds[i : min(n, i + bs)]

# into

for xb, yb = in train_dl:

    
# Dataloader


class DataLoader():
    def __init__(self, ds, bs): 
        self.ds, self.bs = ds, bs
        
    def __iter__(self):
        # Whenever we iterate over our dataloader we can have it handle the slicing of data for us
        for i in range(0, len(self.ds), self.bs): 
            yield self.ds[i:i+self.bs]
            
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
train_dl = DataLoader(train_ds, bs)
valid_dl = DataLoader(valid_ds, bs)

SyntaxError: invalid syntax (937330537.py, line 8)

In [73]:
# Random Sampling - training set should be randomised (not validation set)
# Create a sampler

class Sampler():
    def __init__(self, ds, shuffle = False):
        self.n, self.shuffle = len(ds), shuffle
        
    def __iter__(self):
        res = list(range(self.n))
        if self.shuffle:
            random.shuffle(res)
        return iter(res)

In [74]:
res = list(range(5))
res

[0, 1, 2, 3, 4]

In [75]:
iter(res)

In [76]:
iter(random.shuffle(res))

NameError: name 'random' is not defined

In [77]:
# Batch sampler will provide the batches
# so that we can get a batched randomised set of values
class BatchSampler():
    def __init__(self, sampler, bs, drop_last=False): 
        fc.store_attr()
    def __iter__(self): 
        yield from fc.chunked(iter(self.sampler), self.bs, drop_last=self.drop_last)

In [78]:
# Multiprocessing Dataloader - that runs far faster and also does some multiprocessing

import torch.multiprocessing as mp
from fastcore.basics import store_attr

# they split up multiple workers and make them work across multiple batches

# Sampler into dataloader with a dataset
train_samp = BatchSampler(RandomSampler(train_ds),     bs, drop_last=False)
valid_samp = BatchSampler(SequentialSampler(valid_ds), bs, drop_last=False)

train_dl = DataLoader(train_ds, batch_sampler=train_samp, collate_fn=collate)
valid_dl = DataLoader(valid_ds, batch_sampler=valid_samp, collate_fn=collate)

NameError: name 'RandomSampler' is not defined

In [79]:
# How to get hugging face datasets


import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close

In [80]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [81]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [82]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296607, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233810, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}

In [83]:
dsd = load_dataset(name)
dsd
# dataset dictionary taht contains train and test items, thbey are split into training and testing 

Found cached dataset fashion_mnist (C:/Users/diogo/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [84]:
train, test = dsd['train'],dsd['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [85]:
x,y = ds_builder.info.features

In [86]:
x,y

('image', 'label')

In [87]:
x,y = 'image','label'
img = train[0][x]
img
     

In [88]:
featy = train.features[y]
featy

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [89]:

xb = train[:5][x]
yb = train[:5][y]
yb
     
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [90]:
def collate_fn(b):
    return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
            y:tensor([o[y] for o in b])}

# can return a dictionary of both parts as a collate function

In [91]:
# Dataloader of train, collate_function and the nbathc size of 16
dl = DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [92]:
def transforms(b):
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [93]:
# nbdev allows you to create modules from your noteboooks
#|export - this will put this notebook into a python module
#| default_exp datasets

In [95]:
# Learners are very general purpose training loops and they will use callbacks
# callbacks are called at the end of each epoch / end of each calculation
# if cb: cb(i)

# partial() allows you to create a function that has not run yet, is just partial

# Callables can be made by classes too

def g(a,b,c=0): print(a,b,c)
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3
